# ChewBBACA tests

Uitvoeren van ChewBBACA op test-data (niet- tutorial)
https://github.com/B-UMMI/chewBBACA


### SUMMARY of steps
1) if necessary: create training file with prodigal
2) Scheme creation
    - do wgMLST on reference assemblies
    - allele calling on the >> loci selected (with reference assemblies)
    - Remove paralogs
    - test quality
    - Select loci for 95% in reference assembly (cgMLST) = CORE
3) Allele calling assemblies of interest
4) Optional: include 

### 1) Make sure everything is installed correctly

In [ ]:
#isntall chewbbaca

sudo pip3 install --user chewbbaca
#not complete?

pip install chewbbaca

In [ ]:
#INSTALLING (only once) python modules
pip3 install --user numpy
pip3 install --user scipy
pip3 install --user  biopython
pip3 install --user plotly
pip3 install --user SPARQLWrapper

In [ ]:
#installing dependancies:
#blastp
sudo rpm -Uhv ncbi-blast-2.10.0+-4.x86_64.rpm

#prodigal
conda install -c bioconda prodigal

#extra

#clustalW2
#get install tar.file: http://www.clustal.org/download/current/
tar zvxf clustalw-2.1-linux-x86_64-libcppstatic.tar.gz 
rm clustalw-2.1-linux-x86_64-libcppstatic.tar.gz 
sudo mv clustalw-2.1-linux-x86_64-libcppstatic/ /opt/
sudo ln -s /opt/clustalw-2.1-linux-x86_64-libcppstatic/ /usr/local/bin/clustalw

#mafft
#get rpm file: https://mafft.cbrc.jp/alignment/software/linux.html
sudo rpm -Uvh mafft-7.464-gcc_fc6.x86_64.rpm


In [13]:
#Checking versions

#chewBBACA 
chewBBACA.py --version
#prodigal >=2.6.0
prodigal -v
#blast+
blastp -version
#python
python --version

#numpy >=1.14.0
python -c "import numpy; print(numpy.__version__)"
#scipy >=0.13.3
python -c "import scipy; print(scipy.__version__)"
#biopython >=1.7
python -c "import Bio; print(Bio.__version__)"
#plotly >=1.12.9
python -c "import plotly; print(plotly.__version__)"
#SPARQLWrapper >=1.8.0
python -c "import SPARQLWrapper; print(SPARQLWrapper.__version__)"


2.0.17.2

Prodigal V2.6.3: February, 2016

blastp: 2.9.0+
 Package: blast 2.9.0, build Sep 28 2019 00:38:34
Python 3.7.4
1.18.1
1.4.1
1.76
4.5.3
1.8.5


-> Ok, all installations complete

## 2) Get species specific training files (with prodigal)
This is needed as optional parameters for scheme creation

First look if the files are available for the species your looking at

In [15]:
cd ~/PROJECTHH
#git clone https://github.com/mickaelsilva/prodigal_training_files
#folder created in PROJECTHH "prodigal_training_files"
cd prodigal_training_files
mkdir mickaelsilva_training_files
mv *.trn README.md mickaelsilva_training_files

Cloning into 'prodigal_training_files'...
remote: Enumerating objects: 27, done.
remote: Total 27 (delta 0), reused 0 (delta 0), pack-reused 27
Unpacking objects: 100% (27/27), done.



OR create a new training file

In [ ]:
#OUTDATED: reference genome see getting assembled data

#get a reference genome
mkdir referenceGenomes
#BASH: pip install ncbi-genome-download
#get all bacteria refseq genomes:
ncbi-genome-download bacteria
#or download it yourself e.g. https://www.ncbi.nlm.nih.gov/genome/815?genome_assembly_id=168877

# create your own training file based on the refseq genome you want
# possibility: script to search for this RefSEq number (?)
#e.g. K. pneumoniae = https://www.genome.jp/dbget-bin/www_bget?refseq+NC_009653 = GCF_000016305.1
#BASH: prodigal -i ~/PROJECTHH/prodigal_training_files/k-pneumoniae.fasta -t Klebsiella_pneumoniae.trn -p single

In [ ]:
#Create training file in prodigal trianing mode

cd /home/PROJECTHH/Tools/prodigal

#example with 1 input file: prodigal -i genome1.fna -p train -t genome1.trn
prodigal -i refDB/GCF_000364385.3.fa.gz -p train -t txid573.trn
#invalid meta/singe genome type specified : unzip!
gunzip refDB/GCF_000364385.3.fa.gz
#retry1:
prodigal -i refDB/GCF_000364385.3.fa -p train -t txid573.trn
#Fail: input should be multiple fasta

#make multifasta with 5 ref genomes
#manually selected 5 but can be done with head/...
gunzip *.fa.gz #unzip them
cat GCF*.fa > multi5-txid573.fa #concatenate them

#retry2
prodigal -i refDB/multi5-txid573.fa -p train -t txid573.trn
#still doesn't work, the train mode is written down in online communication
#but not if doing prodigal -h -> only choices are meta and single

#retry3
prodigal -i refDB/multi5-txid573.fa -p meta -t txid573.trn
#starts but cannot continue because not TRAINING mode 

#retry4 SUCCES
prodigal -i refDB/multi5-txid573.fa -p single -t txid573.trn
#starts and finishes :)


Remark: here were 5 full genoms used to create the training file (.trn) is not nearly enogh: should use >100!

OUTPUT:
- txid573.trn

### 3) Set up folder


In [ ]:
cd ~/PROJECTHH
mkdir chewBBACA_run1
cd chewBBACA_run1
mkdir Genomes
mkdir schema #remove

FOUT!! map schema mag nog niet bestaan!!!

### 4) Check if fasta-files are in unix format


Not necessary because no problem in practice

### 5)ALTERNATIVE: external schema
e.g. https://www.cgmlst.org/ncs/schema/2187931/ -> he FASTA files with the allele sequences for each gene of the schema have to be in a single directory.

In [ ]:
#convert the external schema 
#BASH: mkdir cgMLST-KP
#BASH: cd cgMLST-KP
#BASH: unzip ~/Downloads/wfx_mks_1.0.8.1.zip 
#BASH: chewBBACA.py PrepExternalSchema -i ~/PROJECTHH -o ~/PROJECTHH/chewBBACA_run1/schema/existingSchema

### X Getting Full genomes for scheme creation
### OUTDATED: see getting assembly data

use first 20 of downloaded assemblies but for better results go up to 500!!!

In [2]:
mkdir ~/PROJECTHH/dataKP-assembly/KP-testset20
cd ~/PROJECTHH/dataKP-assembly/390KP_assem_fasta
find *.fna.gz |head -20| xargs cp -t ~/PROJECTHH/dataKP-assembly/KP-testset20

find: ‘standard output’: Broken pipe
find: schrijffout


In [ ]:
#unzip the fasta files
#BASH: gunzip *

### 6) wgMLST scheme creation
To find all loci in the defined genomes (all: this means many loci are defined)
-> output = fasta files van alle loci (bvb 500bp lang, of 800bp lang) per proteine en per genoom (bvb. +/- 400 files per genoom)

Time for 20 KP genomes:+/- 55min

In [ ]:
%%bash

#first try with custum trn file
chewBBACA.py CreateSchema -i ~/PROJECTHH/dataKP-assembly/KP-testset20/ \
-o ~/PROJECTHH/chewBBACA_run1/schema --cpu 3 --ptf ~/PROJECTHH/prodigal_training_files/Klebsiella_pneumoniae.trn

remark first try: error after running 40min : [Errno 17] File exists: '/home/hannelore/PROJECTHH/chewBBACA_run1/schema'


In [ ]:
#retry1 (with own .trn file):
chewBBACA.py CreateSchema -i Genomes-20/ -o schema --cpu 2 --ptf txid573.trn
#finished too quickly (?) -> gunzip the reference genomes
gunzip Genomes-20/GCF*

#retry2
chewBBACA.py CreateSchema -i Genomes-20/ -o schema --cpu 2 --ptf txid573.trn
#succes


Starting Script at : 11:35:09-26/04/2020
Finished Script at : 12:31:04-26/04/2020


OUTPUT:
- defined folder (e.g. schema) with >> fasta files (the loci)
- proteinID_Genome.tsv (= list of protein 1-... and the position in the genome)

### 8) Allele calling for scheme optimalisation
on the full genomes (not your data of analysis) -> to find the alleles that are present in all the genomes for the loci defined

In [ ]:
%%bash

chewBBACA.py AlleleCall -i Genomes-20/ -g schema/ -o allelecalling/ --cpu 3 --ptf txid573.trn

+/- 1 hour for 20 genomes KP

OUTPUT:
seperate folder in defined output folder with:
- logging_info.txt
- RepeatedLoci.txt
- results_alleles.tsv
- results_contigsInfo.tsv
- results_statistics.tsv

In [ ]:
#to look at files install libreoffice
#download
#extract
#go into folder -> untill RPMS
sudo dnf install *.rpm


### 9) Paralog detection for scheme optimalisation -> cgMLST scheme
check if some of the loci can be considered paralogs and get loci that can be found in 95% of the analysed genomes (= cgMLST)

In [ ]:
#removegenes
chewBBACA.py RemoveGenes -i allelecalling/results_20200427T144805/results_alleles.tsv \
-g allelecalling/results_20200427T144805/RepeatedLoci.txt -o alleleCallMatrix_cg


#testgenomequality
chewBBACA.py TestGenomeQuality -i alleleCallMatrix_cg.tsv -n 13 -t 200 -s 5


#take loci only if present in 95% for cgMLST

OUTPUT removegenes:
- alleleCallMatrix_cg.tsv (de matrix met verwijderde paralogen)

OUTPUT testgenomequality
- file:///home/hannelore/PROJECTHH/Tools/chewBBACA_run1/GenomeQualityPlot.html
- Genes_95%.txt (list of all the genes)
- RemovedGenomes.txt
    
->  3859 loci present in 95% of genomes  (+/-19/20)

### 10) Remove auxillary genome
op basis van allelecallmatrix_cg file gegenereerd
-p = 95% (=cgMLST = in 95% van de testgenomes aanwezig)

In [ ]:
%%bash

#Select a subset of loci without missing data
chewBBACA.py ExtractCgMLST -i alleleCallMatrix_cg.tsv -o cgMLST_completegenomes -p 0.95


OUTPUT in command line
deleted: 4841 loci
total loci remaining: 3859

OUTPUT:
Folder with
- cgMLSTschema.txt
- cgMLST.tsv
- mdata_stats.tsv
- Presence_Absence.tsv

### 11) Allele call for assemblies of interest 
= actual analysis
- as input took 2 executed assemblies (megahit) and 10 new assemblies from the downloaded genomes NCBI = input (txid 573)

In [ ]:
#use Genes_95%.txt but all loci should have full path to the fasta file

#try1
cp Genes_95%.txt to scheme folder 
#then execute allele call 
chewBBACA.py AlleleCall -i .input/ -g schema/Genes_95%.txt -o results --cpu 3 --ptf txid573.trn
#FAIL

#should add "/home/hannelore/PROJECTHH/Tools/chewBBACA_run1/schema/" to all in txt file
#write script


In [ ]:
#python script that adds extension to all elements of txt file
#see Scripts: Adjusted_genes95%.py)

#!/usr/bin/env python3
def Addpath (file, path):
    fil = open (file, 'r')
    output = open("Genes_95%-PATH.txt", 'w+')
    line_count = 1
    for line in fil:
        for word in line.split():
            if(line_count>3):
                output.write("{}{} ".format(path,word))
            line_count += 1

    output.close()
    fil.close()
    
#test: 
Addpath("Genes_95%.txt", "/home/hannelore/PROJECTHH/Tools/chewBBACA_run1/schema/")
#OK

In [ ]:
#retry1
chewBBACA.py AlleleCall -i input/ -g Genes_95%-PATH.txt -o results --cpu 3 --ptf txid573.trn
#FAIL
#ERROR! No usable gene files in Genes_95%-PATH.txt

#adjusted script to add \n (see Scripts: Adjusted_genes95%.py)
#retry2
chewBBACA.py AlleleCall -i input/ -g Genes_95%-PATH.txt -o results --cpu 3 --ptf txid573.trn
#programme start but won't use files because:
# ERROR! These loci have no short gene file: ['/home/hannelore/PROJECTHH/Tools/chewBBACA_run1/schema/5']

#retry3
chewBBACA.py AlleleCall -i input/ -g schema/ -o results --cpu 3 --ptf txid573.trn
#succes! script was not necessary... 

-> took +/- 30 minutes for 12 samples

OUTPUT:
seperate folder in defined output folder with:
- logging_info.txt
- RepeatedLoci.txt
- results_alleles.tsv
- results_contigsInfo.tsv
- results_statistics.tsv

In [ ]:
#Above FULL scheme was used, only need loci that are 95% present (<<) 
#-> wrote script to move only the fasta files in the CORE -> less time!

#!/usr/bin/env python3

import os
import shutil

inputtie = '/home/hannelore/PROJECTHH/Scripts/tests/test-input/'
outputtie = '/home/hannelore/PROJECTHH/Scripts/tests/test-output/'
fileList = os.listdir(inputtie)

test= inputtie + outputtie
print(test)

cfil = open ('/home/hannelore/PROJECTHH/Scripts/tests/tocopy.txt', 'r')
for line in cfil:
    #Had problems matching line to file: but found that the number of characters differed (1 extra for line)
    #tried to remove with strip and replace (" ","") but this did not work, removing the last character did, it was an /n
    """
    line.split()
    line.replace(" ","")
    """
    line=line[:-1]
    
    for item in fileList:
        print("line is", line)
        print(type(line))
        print(len(line))
        print("item is", item)
        print(type(item))
        print(len(item))

        if(line == item):

            inputpath= str(inputtie) + str(line)
            print(inputpath)
            outputpath= str(outputtie) + str(line)
            print(outputpath)
            shutil.copyfile(str(inputpath), str(outputpath))


cfil.close()


### 12) concatenate & quality
put together matrix with control genomes and results of interested genomes
- Interested genomes: results >... > results_alleles.tsv
- Control genomes: cgMLST_completegenoems > cgMLST.tsv

remark: columns are not exactly the same in cgMLST.tsv 5% of columns/loci have been removed

In [11]:
#write script in python (see script: concatenate-alleles-cgMLST.py)
import os
os.chdir("/home/hannelore/PROJECTHH/Tools/chewBBACA_run1")
def concat (file1, file2):
    import pandas as pd #does work in terminal
    fil1 = pd.read_csv(file1, sep='\t', header=0)
    fil2 = pd.read_csv(file2, sep='\t', header=0)
    files = [fil1, fil2]
    combine = pd.concat(files, join="inner")
    combine.to_csv("finalMatrix_cg.tsv", sep='\t')
   

concat("/home/hannelore/PROJECTHH/Tools/chewBBACA_run1/cgMLST_completegenomes/cgMLST.tsv", "/home/hannelore/PROJECTHH/Tools/chewBBACA_run1/results/results_20200427T171943/results_alleles.tsv")

In [ ]:
%%bash
# assess the cgMLST allele quality attribution for all the genomes

chewBBACA.py TestGenomeQuality -i finalMatrix_cg.tsv -n 13 -t 300 -s 5

ERROR : not compatible

chewBBACA version 2.1.0 by Mickael Silva at https://github.com/B-UMMI/chewBBACA
email contact: mickaelsilva@medicina.ulisboa.pt
will try to open file...

PendingDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.

generating presence abscence matrix
too many indices for array


-> Adjust SCript to remove first column (with numbers)

#adjusted the script:
import os
os.chdir("/home/hannelore/PROJECTHH/Tools/chewBBACA_run1")
def concat (file1, file2):
    import pandas as pd #does work in terminal
    fil1 = pd.read_csv(file1, sep='\t', header=0)
    fil2 = pd.read_csv(file2, sep='\t', header=0)
    files = [fil1, fil2]
    combine = pd.concat(files, join="inner", ignore_index=True)
    #combineD = combine.drop(combine.columns[0], axis=1)
    #if deleting column 0 = FILE, how to delete -1?
    combine.to_csv("finalMatrix2_cg.tsv", sep='\t', index=False) #added index=False

concat("/home/hannelore/PROJECTHH/Tools/chewBBACA_run1/cgMLST_completegenomes/cgMLST.tsv", "/home/hannelore/PROJECTHH/Tools/chewBBACA_run1/results/results_20200427T171943/results_alleles.tsv")

In [ ]:
%%bash
# assess the cgMLST allele quality attribution for all the genomes try2

chewBBACA.py TestGenomeQuality -i finalMatrix2_cg.tsv -n 13 -t 300 -s 5

#OUTPUT:
in current folder:
    - Genes_95%.txt
    - GenomeQualityPlot.html
    - removedGenomes.txt


### 13) Save the cgMLST schema
Select a treshold for further analysis (arbitrary)

here selected treshhold= 25

The following command creates a directory analysis_all/cgMLST_25/ and saves the cgMLST schema selected at the chosen threshold to the file cgMLST.tsv.

In [ ]:
#make file with genomes with treshhold <=25 ???????why?????

In [2]:
%%bash
chewBBACA.py ExtractCgMLST -i finalMatrix2_cg.tsv -o cgMLST_noT -g removedGenomes.txt
 #deleted : 401 loci
 #total loci remaining : 3458


chewBBACA version 2.1.0 by Mickael Silva at https://github.com/B-UMMI/chewBBACA
email contact: mickaelsilva@medicina.ulisboa.pt
building the presence and absence matrix...
presence and abscence matrix built
processing the matrix
deleted : 401 loci
total loci remaining : 3458


### 14) MInimal spanning Tree
Use phyloviz online : online tool but also 
alternative: GrapeTree, MSTMap

In [3]:
#make metadata table (tab separated txt file, e.g. csv/tsv)




In [ ]:
## GRAPETREE
#install GrapeTree  https://github.com/achtman-lab/GrapeTree
pip3 install --user grapetree

In [6]:
%%bash
#GrapeTree execution
cd /home/hannelore/PROJECTHH/Tools/chewBBACA_run1/


#profile = tsv file, with # starting first row (the column-names) = -p and must be tab-delimited text file
#change cgMLST.tsv -> cgMLST-test.profile = txt format
#grapetree -p cgMLST_noT/cgMLST-test.profile -m MSTreeV2
#fail
grapetree -p cgMLST_noT/cgMLST-test.profile -m NJ
#result (see below)

#metadata (tsv or csv), column with ID or first column for ID names = ????

#distance matrix
grapetree -p cgMLST_noT/cgMLST-test.profile -m distance

(((((((((((GCF_001455995.1.fa:2.65385,GCF_001644765.1.fa:3.34615):13.38,GCF_001648215.1.fa:16.62):8.88542,GCF_001701425.2.fa:26.1146):15.0163,GCF_001663435.1.fa:35.7337):442.652,GCF_001482345.1.fa:487.598):6.78454,GCF_000775955.1.fa:481.84):4.34635,GCF_000968155.1.fa:490.841):3.48682,(((GCF_001307175.1.fa:235.25,(GCF_001663195.1.fa:12.5536,GCF_000764615.1.fa:12.4464):222.75):244.581,(GCF_001676825.1.fa:0.5,GCF_001646625.1.fa:0.5):483.669):8.67004,GCF_001663455.1.fa:493.33):1.96631):6.27682,GCF_001548315.1.fa:522.59):10.9718,(GCF_001456055.3.fa:2,GCF_001456095.3.fa:2):477.11):19.0137,((((((((ERR1795072.contigs.fa:6.19519,GCF_001702045.2.fa:11.8048):0.107544,(ERR1795073.contigs.fa:11.4388,GCF_001701845.2.fa:5.5612):0.642456):7.58801,GCF_000807395.2.fa:13.287):4.9198,(GCF_001022235.1.fa:9.25106,GCF_001675125.1.fa:18.7489):4.75239):5.53911,GCF_000814305.1.fa:23.2833):68.8192,((GCF_000364385.3.fa:27.8564,GCF_001022035.1.fa:23.1436):0.729167,GCF_001701835.2.fa:28.2708):21.5183):126.221,GCF_0

#visualiseren in R???? 
- https://rdrr.io/cran/igraph/man/mst.html
- https://systematicinvestor.github.io/Minimum-Spanning-Tree 

In [2]:
#PHyloviz install
#download from: https://bitbucket.org/phyloviz/phyloviz-main/downloads/
unzip phyloviz-2.0a.zip
#mv to opt
sudo mv phyloviz /opt/
#create link
sudo ln -s /opt/phyloviz/phyloviz.exe /usr/local/bin/phyloviz #werkt niet...

SyntaxError: invalid syntax (<ipython-input-2-4166ca27eaf0>, line 3)

In [ ]:
#MSTmap http://mstmap.org/download.html

#download from website

tar zvxf MSTMap.source.tar.gz 
rm MSTMap.source..tar.gz 
sudo mv MSTmap /opt/
cd /opt/MSTmap
make #to compile
sudo ln -s /opt/MSTmap/MSTmap /usr/local/bin/MSTmap

#INPUT: very specific: generate such a file

In [ ]:
#TODO: https://bioinformaticsdotca.github.io/GenEpi_2017_module3_lab 
#good tutorial!!!

### Scheme evaluation

https://github.com/B-UMMI/chewBBACA/wiki/4.-Schema-Evaluation

## PROBLEM WITH ALLELECALL with v2.1.0
See issue62: https://github.com/B-UMMI/chewBBACA/issues/62

In [ ]:
%%bash

#Clone new chewBBACA from master
git clone https://github.com/B-UMMI/chewBBACA.git

#install new environment https://packaging.python.org/guides/installing-using-pip-and-virtual-environments/
python3 -m pip install --user virtualenv
python3 -m venv envPROJECTHH
#activate environment
source envPROJECTHH/bin/activate

#install chewbbaca (from cloned folder)
pip install .

#fixed BUT only .fasta accepted, not .fa -> asked to developers: was fixed! -> clone and install again

#some problems with compatibility of the schemes-> prep them
chewBBACA.py PrepExternalSchema

#as a result problems with matplotlib -> also install in the envPROJECTHH environment
#as a result problems with training file (not compatible)
#when performing allelcall I can not click 


-> I posted this on Github page: https://github.com/B-UMMI/chewBBACA/issues/62
It was reported as a bug, I was suggested to download v2.5.0 in new environment which worked but gave some new compatibility issues with the workflow...

## PROBLEM ALLELCALL with v2.5.0
Problem remains, especially a problem for scheme creation

In [ ]:
#remove old version
pip uninstall chewbbaca
#try to fix it by installing v2.5.4  (13/08/2020)
#Clone new chewBBACA from master
git clone https://github.com/B-UMMI/chewBBACA.git
#in folder (with setup file)
pip install .

In [ ]:
#-> new problem with v2.5.4

### Reinstall v2.5.0

In [ ]:
pip install 